# Introduction

Le concours "Future Sales" est l'évaluation finale du cours "How to win a Data Science" de la spécialisation "Advanced Machine Learning" de l'université HSE de Moscou. L'objectif est de prédire les ventes mensuelles d'articles dans des magasins spécifiques, à partir de données historiques. Le nombre de ventes est compris entre 0 et 20.

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.base import clone
from sklearn.model_selection import GridSearchCV
sns.set(style="darkgrid")

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Chargement des données

In [ ]:
# load data
items=pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/items.csv")
shops=pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/shops.csv")
cats=pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/item_categories.csv")
train=pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv")
test=pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/test.csv")

# 1. Nettoyage des données

Suppression des outliers nettoyage des données brute et ajout de variables

## Exploration des données

In [ ]:
print(f'items.csv : {items.shape}')
items.info()

In [ ]:
print(f'item_categories.csv : {cats.shape}')
cats.info()

In [ ]:
print(f'shops.csv : {shops.shape}')
shops.info()

In [ ]:
print(f'sales_train.csv : {train.shape}')
train.info()

In [ ]:
print(f'test.csv : {test.shape}')
test.info()

## Remove Nan

In [ ]:
train.isnull().sum()

## Suppression des outliers

In [ ]:
plt.figure(figsize=(10, 4))
plt.xlim(-100, 3000)
flierprops = dict(marker='o', markerfacecolor='purple', markersize=6,
                  linestyle='none', markeredgecolor='black')
sns.boxplot(x=train.item_cnt_day, flierprops=flierprops)

plt.figure(figsize=(10, 4))
plt.xlim(train.item_price.min(), train.item_price.max()*1.1)
sns.boxplot(x=train.item_price, flierprops=flierprops)

Nous allons supprimer les aberrations évidentes de l'ensemble de données - les articles vendus à plus de 1000 exemplaires en un jour et les articles dont le prix est supérieur à 300 000.

In [ ]:
train = train[(train.item_price < 300000) & (train.item_cnt_day < 1000)]

Supprimez toutes les lignes du train où le prix de l'article est négatif.  
Pour le nombre d'item vendu il peut s'agir de remboursements on mettra les valeurs inférieur à 1 à zéro afin de supprimer les valeurs négatives.

In [ ]:
train = train[train.item_price > 0].reset_index(drop=True)
train.loc[train.item_cnt_day < 1, "item_cnt_day"] = 0

## Nettoyage Shop Data

Plusieurs magasins semblent être des doublons les uns des autres. Cela pourrait être dû à la réouverture de magasins ou à un changement d'emplacement dans la même rue ou le même centre commercial.

In [ ]:
# Якутск Орджоникидзе, 56
train.loc[train.shop_id == 0, 'shop_id'] = 57
test.loc[test.shop_id == 0, 'shop_id'] = 57
# Якутск ТЦ "Центральный"
train.loc[train.shop_id == 1, 'shop_id'] = 58
test.loc[test.shop_id == 1, 'shop_id'] = 58
# Жуковский ул. Чкалова 39м²
train.loc[train.shop_id == 10, 'shop_id'] = 11
test.loc[test.shop_id == 10, 'shop_id'] = 11

Nettoyer le nom de certain magasin et ajouté une variable ville du magasin.

In [ ]:
shops.loc[shops.shop_name == 'Сергиев Посад ТЦ "7Я"',
          "shop_name"] = 'СергиевПосад ТЦ "7Я"'
shops["city"] = shops.shop_name.str.split(" ").map(lambda x: x[0])
shops["category"] = shops.shop_name.str.split(" ").map(lambda x: x[1])
shops.loc[shops.city == "!Якутск", "city"] = "Якутск"

On ne garde que les catégories de magasin où il y a plus de 5 magasin différents, le reste est regroupé dans autre.

In [ ]:
category = []
for cat in shops.category.unique():
    if len(shops[shops.category == cat]) >= 5:
        category.append(cat)
shops.category = shops.category.apply(
    lambda x: x if (x in category) else "other")

In [ ]:
from sklearn.preprocessing import LabelEncoder
shops["shop_category"] = LabelEncoder().fit_transform(shops.category)
shops["shop_city"] = LabelEncoder().fit_transform(shops.city)
shops = shops[["shop_id", "shop_category", "shop_city"]]

## Nettoyage des catégories d'objet

In [ ]:
cats["type_code"] = cats.item_category_name.apply(
    lambda x: x.split(" ")[0]).astype(str)
cats.loc[(cats.type_code == "Игровые") | (
    cats.type_code == "Аксессуары"), "category"] = "Игры"

In [ ]:
category = []
for cat in cats.type_code.unique():
    if len(cats[cats.type_code == cat]) >= 5:
        category.append(cat)
cats.type_code = cats.type_code.apply(
    lambda x: x if (x in category) else "etc")

In [ ]:
cats.type_code = LabelEncoder().fit_transform(cats.type_code)
cats["split"] = cats.item_category_name.apply(lambda x: x.split("-"))
cats["subtype"] = cats.split.apply(
    lambda x: x[1].strip() if len(x) > 1 else x[0].strip())
cats["subtype_code"] = LabelEncoder().fit_transform(cats["subtype"])
cats = cats[["item_category_id", "subtype_code", "type_code"]]

## Nettoyage des objets

In [ ]:
import re


def name_correction(x):
    x = x.lower()  # all letters lower case
    x = x.partition('[')[0]  # partition by square brackets
    x = x.partition('(')[0]  # partition by curly brackets
    x = re.sub('[^A-Za-z0-9А-Яа-я]+', ' ', x)  # remove special characters
    x = x.replace('  ', ' ')  # replace double spaces with single spaces
    x = x.strip()  # remove leading and trailing white space
    return x

## Nettoyage des noms d'objet

In [ ]:
# split item names by first bracket
items["name1"], items["name2"] = items.item_name.str.split("[", 1).str
items["name1"], items["name3"] = items.item_name.str.split("(", 1).str

# replace special characters and turn to lower case
items["name2"] = items.name2.str.replace(
    '[^A-Za-z0-9А-Яа-я]+', " ").str.lower()
items["name3"] = items.name3.str.replace(
    '[^A-Za-z0-9А-Яа-я]+', " ").str.lower()

# fill nulls with '0'
items = items.fillna('0')

items["item_name"] = items["item_name"].apply(lambda x: name_correction(x))

# return all characters except the last if name 2 is not "0" - the closing bracket
items.name2 = items.name2.apply(lambda x: x[:-1] if x != "0" else "0")

## Nettoyage des types d'objet

In [ ]:
items["type"] = items.name2.apply(lambda x: x[0:8] if x.split(" ")[
                                  0] == "xbox" else x.split(" ")[0])
items.loc[(items.type == "x360") | (items.type == "xbox360")
          | (items.type == "xbox 360"), "type"] = "xbox 360"
items.loc[items.type == "", "type"] = "mac"
items.type = items.type.apply(lambda x: x.replace(" ", ""))
items.loc[(items.type == 'pc') | (items.type == 'pс')
          | (items.type == "pc"), "type"] = "pc"
items.loc[items.type == 'рs3', "type"] = "ps3"

In [ ]:
group_sum = items.groupby(["type"]).agg({"item_id": "count"})
group_sum = group_sum.reset_index()
drop_cols = []
for cat in group_sum.type.unique():
    if group_sum.loc[(group_sum.type == cat), "item_id"].values[0] < 40:
        drop_cols.append(cat)
items.name2 = items.name2.apply(lambda x: "other" if (x in drop_cols) else x)
items = items.drop(["type"], axis=1)

In [ ]:
items.name2 = LabelEncoder().fit_transform(items.name2)
items.name3 = LabelEncoder().fit_transform(items.name3)

items.drop(["item_name", "name1"], axis=1, inplace=True)
items.head()

# 2. Preprocessing

Créez une matrice df avec chaque combinaison de mois, magasin et article par ordre croissant de mois. Item_cnt_day est sommé par mois dans item_cnt_month.

In [ ]:
from itertools import product

matrix = []
cols = ["date_block_num", "shop_id", "item_id"]
for i in range(34):
    sales = train[train.date_block_num == i]
    matrix.append(np.array(list(
        product([i], sales.shop_id.unique(), sales.item_id.unique())), dtype=np.int16))

matrix = pd.DataFrame(np.vstack(matrix), columns=cols)
matrix["date_block_num"] = matrix["date_block_num"].astype(np.int8)
matrix["shop_id"] = matrix["shop_id"].astype(np.int8)
matrix["item_id"] = matrix["item_id"].astype(np.int16)
matrix.sort_values(cols, inplace=True)

In [ ]:
group = train.groupby(["date_block_num", "shop_id", "item_id"]).agg(
    {"item_cnt_day": ["sum"]})
group.columns = ["item_cnt_month"]
group.reset_index(inplace=True)
matrix = pd.merge(matrix, group, on=cols, how="left")
matrix["item_cnt_month"] = matrix["item_cnt_month"].fillna(
    0).astype(np.float16)

Le 34em mois est utilisé comme test set.

In [ ]:
test["date_block_num"] = 34
test["date_block_num"] = test["date_block_num"].astype(np.int8)
test["shop_id"] = test.shop_id.astype(np.int8)
test["item_id"] = test.item_id.astype(np.int16)

Regrouper train et test sets.

In [ ]:
matrix = pd.concat([matrix, test.drop(["ID"], axis=1)],
                   ignore_index=True, sort=False, keys=cols)
matrix.fillna(0, inplace=True)

Ajouter les variables objet, magasin et les catégories à la matrice.

In [ ]:
matrix = pd.merge(matrix, shops, on=["shop_id"], how="left")
matrix = pd.merge(matrix, items, on=["item_id"], how="left")
matrix = pd.merge(matrix, cats, on=["item_category_id"], how="left")
matrix["shop_city"] = matrix["shop_city"].astype(np.int8)
matrix["shop_category"] = matrix["shop_category"].astype(np.int8)
matrix["item_category_id"] = matrix["item_category_id"].astype(np.int8)
matrix["subtype_code"] = matrix["subtype_code"].astype(np.int8)
matrix["name2"] = matrix["name2"].astype(np.int8)
matrix["name3"] = matrix["name3"].astype(np.int16)
matrix["type_code"] = matrix["type_code"].astype(np.int8)

Feature Engineering

* Ajouter les données des mois précédent.

In [ ]:
# Define a lag feature function
def lag_feature(df, lags, cols):
    for col in cols:
        print(col)
        tmp = df[["date_block_num", "shop_id", "item_id", col]]
        for i in lags:
            shifted = tmp.copy()
            shifted.columns = ["date_block_num",
                               "shop_id", "item_id", col + "_lag_"+str(i)]
            shifted.date_block_num = shifted.date_block_num + i
            df = pd.merge(df, shifted, on=[
                          'date_block_num', 'shop_id', 'item_id'], how='left')
    return df

Ajouter item_cnt_month des 3 mois précédent

In [ ]:
matrix = lag_feature(matrix, [1, 2, 3], ["item_cnt_month"])

Ajouter la moyenne de item_cnt du mois précédents.

In [ ]:
group = matrix.groupby(["date_block_num"]).agg({"item_cnt_month": ["mean"]})
group.columns = ["date_avg_item_cnt"]
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=["date_block_num"], how="left")
matrix.date_avg_item_cnt = matrix["date_avg_item_cnt"].astype(np.float16)
matrix = lag_feature(matrix, [1], ["date_avg_item_cnt"])
matrix.drop(["date_avg_item_cnt"], axis=1, inplace=True)

Ajouter la moyenne de item_cnt du mois précédent par objet

In [ ]:
group = matrix.groupby(['date_block_num', 'item_id']).agg(
    {'item_cnt_month': ['mean']})
group.columns = ['date_item_avg_item_cnt']
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num', 'item_id'], how='left')
matrix.date_item_avg_item_cnt = matrix['date_item_avg_item_cnt'].astype(
    np.float16)
matrix = lag_feature(matrix, [1], ['date_item_avg_item_cnt'])
matrix.drop(['date_item_avg_item_cnt'], axis=1, inplace=True)

 Ajouter les variations de prix entre le prix des dernier mois

In [ ]:
group = train.groupby(["item_id"]).agg({"item_price": ["mean"]})
group.columns = ["item_avg_item_price"]
group.reset_index(inplace=True)

matrix = matrix.merge(group, on=["item_id"], how="left")
matrix["item_avg_item_price"] = matrix.item_avg_item_price.astype(np.float16)


group = train.groupby(["date_block_num", "item_id"]
                      ).agg({"item_price": ["mean"]})
group.columns = ["date_item_avg_item_price"]
group.reset_index(inplace=True)

matrix = matrix.merge(group, on=["date_block_num", "item_id"], how="left")
matrix["date_item_avg_item_price"] = matrix.date_item_avg_item_price.astype(
    np.float16)
lags = [1, 2, 3]
matrix = lag_feature(matrix, lags, ["date_item_avg_item_price"])
for i in lags:
    matrix["delta_price_lag_" + str(i)] = (
        matrix["date_item_avg_item_price_lag_" + str(i)] -
        matrix["item_avg_item_price"]) / matrix["item_avg_item_price"]


def select_trends(row):
    for i in lags:
        if row["delta_price_lag_" + str(i)]:
            return row["delta_price_lag_" + str(i)]
    return 0


matrix["delta_price_lag"] = matrix.apply(select_trends, axis=1)
matrix["delta_price_lag"] = matrix.delta_price_lag.astype(np.float16)
matrix["delta_price_lag"].fillna(0, inplace=True)

features_to_drop = ["item_avg_item_price", "date_item_avg_item_price"]
for i in lags:
    features_to_drop.append("date_item_avg_item_price_lag_" + str(i))
    features_to_drop.append("delta_price_lag_" + str(i))
matrix.drop(features_to_drop, axis=1, inplace=True)

Ajouter le mois du premier objet vendu pour chaque magasin et pour chaque objet

In [ ]:
matrix["item_shop_first_sale"] = matrix["date_block_num"] - \
    matrix.groupby(["item_id", "shop_id"])["date_block_num"].transform('min')
matrix["item_first_sale"] = matrix["date_block_num"] - \
    matrix.groupby(["item_id"])["date_block_num"].transform('min')

Ajouter le mois

In [ ]:
matrix["month"] = matrix["date_block_num"] % 12

Supprimer les 3 dernier mois de la matrice. Ils n'ont pas assez de mois précédent.

In [ ]:
matrix = matrix[matrix["date_block_num"] > 3]

In [ ]:
matrix.head()

In [ ]:
matrix.info()

# 3. Modelling

In [ ]:
import gc
import pickle
from xgboost import XGBRegressor
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4

In [ ]:
data = matrix.copy()
del matrix
gc.collect()

In [ ]:
data[data["date_block_num"] == 34].shape

Feature selection

In [ ]:
data.drop(["item_id"] , axis=1, inplace=True)

Utilisation du mois 33 pour la validation et 34 pour le test

In [ ]:
X_train = data[data.date_block_num < 33].drop(['item_cnt_month'], axis=1)
Y_train = data[data.date_block_num < 33]['item_cnt_month']
X_valid = data[data.date_block_num == 33].drop(['item_cnt_month'], axis=1)
Y_valid = data[data.date_block_num == 33]['item_cnt_month']
X_test = data[data.date_block_num == 34].drop(['item_cnt_month'], axis=1)

In [ ]:
Y_train = Y_train.clip(0, 20)
Y_valid = Y_valid.clip(0, 20)

In [ ]:
del group
del train
del cats
del shops
del items
del data
gc.collect()

## Prediction

In [ ]:
model = XGBRegressor(
    objective='reg:squarederror',
    max_depth=10,
    n_estimators=600,
    min_child_weight=0.1,
    subsample=1,
    eta=0.1,
)

model.fit(
    X_train,
    Y_train,
    eval_metric="rmse",
    eval_set=[(X_train, Y_train), (X_valid, Y_valid)],
    verbose=True,
    early_stopping_rounds=20)

In [ ]:
Y_pred = model.predict(X_valid).clip(0, 20)
Y_test = model.predict(X_test).clip(0, 20)

submission = pd.DataFrame({
    "ID": test.index,
    "item_cnt_month": Y_test
})
submission.to_csv('xgb_submission.csv', index=False)

In [ ]:
from xgboost import plot_importance


def plot_features(booster, figsize):
    fig, ax = plt.subplots(1, 1, figsize=figsize)
    return plot_importance(booster=booster, ax=ax)


plot_features(model, (10, 7))